In [ ]:
import pandas as pd
import numpy as np
import io
import re

In [ ]:
# Names in 117th Congress. Taken from wikipedia and editted to match OpenSecrets.org

names = pd.read_csv(io.StringIO('''
Patrick Leahy (D)
Chuck Grassley (R)
Mitch McConnell (R)
Richard C Shelby (R)
Dianne Feinstein (D)
Patty Murray (D)
Ron Wyden (D)
Dick Durbin (D)
Jack Reed (D)
Susan Collins (R)
Charles E Schumer (D)
Mike Crapo (R)
Tom Carper (D)
Debbie Stabenow (D)
Maria Cantwell (D)
John Cornyn (R)
Lisa Murkowski (R)
Lindsey Graham (R)
Richard Burr (R)
John Thune (R)
Robert Menendez (D)
Ben Cardin (D)
Bernie Sanders (I)
Sherrod Brown (D)
Bob Casey (D)
Amy Klobuchar (D)
Sheldon Whitehouse (D)
Jon Tester (D)
John A Barrasso (R)
Roger Wicker (R)
Jeanne Shaheen (D)
Mark Warner (D)
James E Risch (R)
Jeff Merkley (D)
Michael Bennet (D)
Kirsten Gillibrand (D)
Joe Manchin (D)
Chris Coons (D)
Roy Blunt (R)
Jerry Moran (R)
Rob Portman (R)
John Boozman (R)
Pat Toomey (R)
John Hoeven (R)
Marco Rubio (R)
Ron Johnson (R)
Rand Paul (R)
Richard Blumenthal (D)
Mike Lee (R)
Brian Schatz (D)
Tim Scott (R)
Tammy Baldwin (D)
Christopher S Murphy (D)
Mazie K Hirono (D)
Martin Heinrich (D)
Angus King (I)
Tim Kaine (D)
Ted Cruz (R)
Elizabeth Warren (D)
Deb Fischer (R)
Ed Markey (D)
Cory Booker (D)
Shelley Moore Capito (R)
Gary Peters (D)
Bill Cassidy (R)
James Lankford (R)
Tom Cotton (R)
Steven Daines (R)
Mike Rounds (R)
Thom Tillis (R)
Joni Ernst (R)
Ben Sasse (R)
Dan Sullivan (R)
Chris Van Hollen (D)
Todd Young (R)
Tammy Duckworth (D)
Maggie Hassan (D)
Kamala Harris (D)
John Kennedy (R)
Catherine Cortez Masto (D)
Tina Smith (D)
Cindy Hyde-Smith (R)
Marsha Blackburn (R)
Kyrsten Sinema (D)
Kevin Cramer (R)
Jacky Rosen (D)
Mitt Romney (R)
Mike Braun (R)
Josh Hawley (R)
Rick Scott (R)
Mark Kelly (D)
Ben Ray Lujan (D)
Cynthia Lummis (R)
Roger Marshall (R)
John Hickenlooper (D)
Bill Hagerty (R)
Tommy Tuberville (R)
Alex Padilla (D)
Jon Ossoff (D)
Raphael Warnock (D)
'''), header=None)


L = []
for i in range(100):
  L.append(names[0][i])

# separate names and party
names = [name[:-4] for name in L]
party = [name[-3:] for name in L]

# create dictionary mapping names to party
name_to_party = dict(zip(names, party))

In [ ]:
# Checking that names match file names

for name in names:
  try:
    df = pd.read_csv(f"{name}.csv")
  except:
    print(f"({name})")

In [ ]:
# Function for reformatting names from "FIRST LAST" to "LAST, FIRST"

def reverse_name(name):
    parts = name.strip().split()
    last = parts[-1]
    first_middle = ' '.join(parts[:-1])
    return f"{last}, {first_middle}"

In [ ]:
# Function for getting rid of white spaces at the end of organization names

def clean_whitespace(text):
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    return text

In [ ]:
# initialize dataframe to store all data
df = pd.DataFrame()
for name in names:
  # create df for specific senator
  temp_df = pd.read_csv(f"{name}.csv")

  # ensure that donors are sorted by total contribution
  temp_df.sort_values('total', ascending = False, inplace= True)

  # take top 100 donors
  temp_df = temp_df[:100]

  # record senator name in dataframe
  temp_df['Senator'] = name

  # concatenate to larger dataframe
  df = pd.concat([df, temp_df], ignore_index = False)

# list party of senator
df['Party'] = df['Senator'].apply(lambda x: name_to_party.get(x))

# extracting relevant columns
df = df[['Senator', 'Party', 'ultorg', 'total', 'pacs', 'indivs']]

# renaming columns
df = df.rename(columns={'Senator': 'Senator', 'ultorg': 'Organization',
                        'total': 'Total', 'pacs' : 'PACs', 'indivs' : 'Individuals'})

# reformatting names
df['Senator'] = df['Senator'].apply(reverse_name)
df.sort_values(by = ['Senator', 'Total'], ascending = [True, False], inplace = True)

# getting rid of whitespace
df['Organization'] = df['Organization'].apply(clean_whitespace)

In [ ]:
df

,Senator,Party,Organization,Total,PACs,Individuals
0,"Baldwin, Tammy",(D),University of Wisconsin,528258,0,528258
1,"Baldwin, Tammy",(D),JStreetPAC,216748,9706,207042
2,"Baldwin, Tammy",(D),Democracy Engine,187684,0,187684
3,"Baldwin, Tammy",(D),League of Conservation Voters,97896,12213,85683
4,"Baldwin, Tammy",(D),State of Wisconsin,91576,0,91576
...,...,...,...,...,...,...
95,"Young, Todd",(R),Hilton Worldwide,26825,2500,24325
96,"Young, Todd",(R),Mulzer Crushed Stone,26750,0,26750
97,"Young, Todd",(R),Pilot Corp,26700,0,26700
98,"Young, Todd",(R),North American Midway Entertainment,26600,0,26600


In [ ]:
df.to_csv('full_opensecrets_data.csv', index = False)